### Installation

In [1]:
pip install -q tensorflow tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\crisp\\Documents\\anaconda3\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [1]:
!pip install tensorflow_datasets

  Using cached promise-2.3-py3-none-any.whl
  Using cached tensorflow_metadata-0.30.0-py3-none-any.whl (47 kB)
  Using cached dill-0.3.3-py2.py3-none-any.whl (81 kB)
  Using cached googleapis_common_protos-1.53.0-py2.py3-none-any.whl (198 kB)


#### Imports

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import tensorflow_datasets as tfds

### Checking datasets

In [4]:
print(tfds.list_builders())

['abstract_reasoning', 'accentdb', 'aeslc', 'aflw2k3d', 'ag_news_subset', 'ai2_arc', 'ai2_arc_with_ir', 'amazon_us_reviews', 'anli', 'arc', 'bair_robot_pushing_small', 'bccd', 'beans', 'big_patent', 'bigearthnet', 'billsum', 'binarized_mnist', 'binary_alpha_digits', 'blimp', 'bool_q', 'c4', 'caltech101', 'caltech_birds2010', 'caltech_birds2011', 'cars196', 'cassava', 'cats_vs_dogs', 'celeb_a', 'celeb_a_hq', 'cfq', 'cherry_blossoms', 'chexpert', 'cifar10', 'cifar100', 'cifar10_1', 'cifar10_corrupted', 'citrus_leaves', 'cityscapes', 'civil_comments', 'clevr', 'clic', 'clinc_oos', 'cmaterdb', 'cnn_dailymail', 'coco', 'coco_captions', 'coil100', 'colorectal_histology', 'colorectal_histology_large', 'common_voice', 'coqa', 'cos_e', 'cosmos_qa', 'covid19sum', 'crema_d', 'curated_breast_imaging_ddsm', 'cycle_gan', 'd4rl_mujoco_ant', 'd4rl_mujoco_halfcheetah', 'dart', 'davis', 'deep_weeds', 'definite_pronoun_resolution', 'dementiabank', 'diabetic_retinopathy_detection', 'div2k', 'dmlab', 'dolp

### Getting data Infomation

In [5]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info
print(info)

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    full_name='rock_paper_scissors/3.0.0',
    description="""
    Images of hands playing rock, paper, scissor game.
    """,
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    data_path='C:\\Users\\crisp\\tensorflow_datasets\\rock_paper_scissors\\3.0.0',
    download_size=Unknown size,
    dataset_size=Unknown size,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
    },
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dataset"
    }""",
)


### Data Preparation

In [6]:
train = tfds.load(name='rock_paper_scissors', split="train")
test = tfds.load(name='rock_paper_scissors', split='test')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling rock_paper_scissors-train.tfrecord...:   0%|          | 0/2520 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling rock_paper_scissors-test.tfrecord...:   0%|          | 0/372 [00:00<?, ? examples/s]

Dataset rock_paper_scissors downloaded and prepared to C:\Users\crisp\tensorflow_datasets\rock_paper_scissors\3.0.0. Subsequent calls will reuse this data.


### Iterating over data
> To iterate over a tensorflow dataset we do it as follows

In [7]:
for data in train:
  print(data['image'], data['label'])
  break

tf.Tensor(
[[[254 254 254]
  [253 253 253]
  [254 254 254]
  ...
  [251 251 251]
  [250 250 250]
  [250 250 250]]

 [[254 254 254]
  [254 254 254]
  [253 253 253]
  ...
  [250 250 250]
  [251 251 251]
  [249 249 249]]

 [[254 254 254]
  [254 254 254]
  [254 254 254]
  ...
  [251 251 251]
  [250 250 250]
  [252 252 252]]

 ...

 [[252 252 252]
  [251 251 251]
  [252 252 252]
  ...
  [247 247 247]
  [249 249 249]
  [248 248 248]]

 [[253 253 253]
  [253 253 253]
  [251 251 251]
  ...
  [248 248 248]
  [248 248 248]
  [248 248 248]]

 [[252 252 252]
  [253 253 253]
  [252 252 252]
  ...
  [248 248 248]
  [247 247 247]
  [250 250 250]]], shape=(300, 300, 3), dtype=uint8) tf.Tensor(2, shape=(), dtype=int64)


### Creating a Numpy data
> We are going to scale our data and convert it to a nummpy array

In [ ]:
train_images = np.array([data['image'].numpy()/255 for data in train])
train_labels =np.array([data['label'].numpy() for data in train])
test_image = np.array([data['image'].numpy()/255   for data in test])
test_labels = np.array([data['label'].numpy() for data in test])

In [ ]:
train_images[0], train_images.shape

### Class Names
0 - Rock

1 - Paper

2 - Scissors

In [ ]:
class_names = np.array(["rock", "paper", "scissor"])

### Creating a NN

In [ ]:
input_shape = train_images[0].shape
input_shape

In [ ]:

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'),
    keras.layers.MaxPool2D((3,3)) ,
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64, (2, 2), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])
model.summary()

### Combiling the Model

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=.0001),
    metrics=["accuracy"],
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

### Fitting the ModeL

In [ ]:
EPOCHS = 5
BATCH_SIZE = 4
VALIDATION_SET = (test_image, test_labels)
history = model.fit(train_images, train_labels, epochs=EPOCHS, validation_data=VALIDATION_SET, batch_size=BATCH_SIZE)

### Model Evaluation Conclusion
Our model is performing perfect. The loss on the train_set is almost 0 as well as the validation loss. The accuracy on the train set is `100%` compared to `83%` accuracy on the test set.

> The model is just overtraining but giving us good results on the validation set.

### Making Predictions


In [ ]:
predictions = model.predict(test_image[:10])
for i, j in zip(predictions, test_labels[:10]):
  print(class_names[np.argmax(i)],"-------->", class_names[j])

### Tunning Hyper Parameters -- Keras-Tunner
* [Docs](https://www.tensorflow.org/tutorials/keras/keras_tuner)


### Installation

In [ ]:
pip install -q -U keras-tuner

### Importing

In [ ]:
import kerastuner as kt

In [ ]:
def model_builder(hp):

  model = keras.Sequential()
  #   we want the model to find the best unit and the activation function for the first layer for us
  model.add(keras.layers.Conv2D(hp.Int('units',  min_value=32, max_value=512, step=32),(3, 3), 
                                input_shape=input_shape, activation=hp.Choice('activation-fn',values=['relu', 'sgd'])))
  
  model.add(keras.layers.MaxPool2D((3,3)))
  model.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
  model.add(keras.layers.MaxPool2D((2,2)))
  model.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
  model.add(keras.layers.MaxPool2D((2,2)))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(64, activation='relu'))
  model.add(keras.layers.Dense(32, activation='relu'))
  model.add(keras.layers.Dense(3, activation='softmax'))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     )

In [ ]:
tuner.search(train_images, train_labels, validation_data=VALIDATION_SET, epochs=EPOCHS, batch_size=BATCH_SIZE)

> That's basically how the `kerastunner` works